# Creating a simple first model
>  In this chapter, you'll build a first-pass model. You'll use numeric data only to train the model. Spoiler alert - throwing out all of the text data is bad for performance! But you'll learn how to format your predictions. Then, you'll be introduced to natural language processing (NLP) in order to start working with the large amounts of text in the data.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 2 exercises "Case Study: School Budgeting with Machine Learning in Python" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (8, 8)

## It's time to build a model

### Setting up a train-test split in scikit-learn

<div class=""><p>Alright, you've been patient and awesome. It's finally time to start training models! </p>
<p>The first step is to split the data into a training set and a test set. Some labels don't occur very often, but we want to make sure that they appear in both the training and the test sets. We provide a function that will make sure at least <code>min_count</code> examples of each label appear in each split: <code>multilabel_train_test_split</code>.</p>
<p>Feel free to check out the full code for <code>multilabel_train_test_split</code> <a href="https://github.com/drivendataorg/box-plots-sklearn/blob/master/src/data/multilabel.py" target="_blank" rel="noopener noreferrer">here</a>.</p>
<p>You'll start with a simple model that uses <strong>just the numeric columns</strong> of your DataFrame when calling <code>multilabel_train_test_split</code>. The data has been read into a DataFrame <code>df</code> and a list consisting of just the numeric columns is available as <code>NUMERIC_COLUMNS</code>.</p></div>

In [10]:
NUMERIC_COLUMNS = ['FTE', 'Total']
LABELS = ['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type', 'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status']

In [4]:
df = pd.read_csv('https://github.com/lnunesAI/Datacamp/raw/main/3-skill-tracks/case-study-school-budgeting-with-machine-learning-in-python/data/TrainingData.csv', index_col=0)

In [17]:
#https://github.com/drivendataorg/box-plots-sklearn/blob/master/src/data/multilabel.py

from warnings import warn

import numpy as np
import pandas as pd

def multilabel_sample(y, size=1000, min_count=5, seed=None):
    """ Takes a matrix of binary labels `y` and returns
        the indices for a sample of size `size` if
        `size` > 1 or `size` * len(y) if size =< 1.
        The sample is guaranteed to have > `min_count` of
        each label.
    """
    try:
        if (np.unique(y).astype(int) != np.array([0, 1])).any():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('multilabel_sample only works with binary indicator matrices')

    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary.')

    if size <= 1:
        size = np.floor(y.shape[0] * size)

    if y.shape[1] * min_count > size:
        msg = "Size less than number of columns * min_count, returning {} items instead of {}."
        warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count

    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))

    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])

    sample_idxs = np.array([], dtype=choices.dtype)

    # first, guarantee > min_count of each label
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])


def multilabel_sample_dataframe(df, labels, size, min_count=5, seed=None):
    """ Takes a dataframe `df` and returns a sample of size `size` where all
        classes in the binary matrix `labels` are represented at
        least `min_count` times.
    """
    idxs = multilabel_sample(labels, size=size, min_count=min_count, seed=seed)
    return df.loc[idxs]


def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    """ Takes a features matrix `X` and a label matrix `Y` and
        returns (X_train, X_test, Y_train, Y_test) where all
        classes in Y are represented at least `min_count` times.
    """
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])

    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])

Instructions
<ul>
<li>Create a new DataFrame named <code>numeric_data_only</code> by applying the <code>.fillna(-1000)</code> method to the numeric columns (available in the list <code>NUMERIC_COLUMNS</code>) of <code>df</code>.</li>
<li>Convert the labels (available in the list <code>LABELS</code>) to dummy variables. Save the result as <code>label_dummies</code>.</li>
<li>In the call to <code>multilabel_train_test_split()</code>, set the <code>size</code> of your test set to be <code>0.2</code>. Use a <code>seed</code> of <code>123</code>.</li>
<li>Fill in the <code>.info()</code> method calls for <code>X_train</code>, <code>X_test</code>, <code>y_train</code>, and <code>y_test</code>.</li>
</ul>

In [18]:
# Create the new DataFrame: numeric_data_only
numeric_data_only = df[NUMERIC_COLUMNS].fillna(-1000)

# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS])

# Create training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(numeric_data_only,
                                                               label_dummies,
                                                               size=0.2, 
                                                               seed=123)
# Print the info
print("X_train info:")
print(X_train.info())
print("\nX_test info:")  
print(X_test.info())
print("\ny_train info:")  
print(y_train.info())
print("\ny_test info:")  
print(y_test.info()) 

X_train info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040 entries, 198 to 101861
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FTE     1040 non-null   float64
 1   Total   1040 non-null   float64
dtypes: float64(2)
memory usage: 24.4 KB
None

X_test info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 209 to 448628
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FTE     520 non-null    float64
 1   Total   520 non-null    float64
dtypes: float64(2)
memory usage: 12.2 KB
None

y_train info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040 entries, 198 to 101861
Columns: 104 entries, Function_Aides Compensation to Operating_Status_PreK-12 Operating
dtypes: uint8(104)
memory usage: 113.8 KB
None

y_test info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 209 to 448628
Columns: 104 entries, Function_Aides Compensat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Size less than number of columns * min_count, returning 520 items instead of 312.0.


### Training a model

<div class=""><p>With split data in hand, you're only a few lines away from training a model.</p>
<p>In this exercise, you will import the logistic regression and one versus rest classifiers in order to fit a multi-class logistic regression model to the <code>NUMERIC_COLUMNS</code> of your feature data.</p>
<p>Then you'll test and print the accuracy with the <code>.score()</code> method to see the results of training. </p>
<p><strong>Before you train!</strong> Remember, we're ultimately going to be using logloss to score our model, so don't worry too much about the accuracy here. Keep in mind that you're throwing away all of the text data in the dataset - that's by far most of the data! So don't get your hopes up for a killer performance just yet. We're just interested in getting things up and running at the moment.</p>
<p>All data necessary to call <code>multilabel_train_test_split()</code> has been loaded into the workspace.</p></div>

Instructions
<ul>
<li>Import <code>LogisticRegression</code> from <code>sklearn.linear_model</code> and <code>OneVsRestClassifier</code> from <code>sklearn.multiclass</code>.</li>
<li>Instantiate the classifier <code>clf</code> by placing <code>LogisticRegression()</code> inside <code>OneVsRestClassifier()</code>.</li>
<li>Fit the classifier to the training data <code>X_train</code> and <code>y_train</code>.</li>
<li>Compute and print the accuracy of the classifier using its <code>.score()</code> method, which accepts two arguments: <code>X_test</code> and <code>y_test</code>.</li>
</ul>

In [19]:
# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Create the DataFrame: numeric_data_only
numeric_data_only = df[NUMERIC_COLUMNS].fillna(-1000)

# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS])

# Create training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(numeric_data_only,
                                                               label_dummies,
                                                               size=0.2, 
                                                               seed=123)

# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Print the accuracy
print("Accuracy: {}".format(clf.score(X_test, y_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Size less than number of columns * min_count, returning 520 items instead of 312.0.


Accuracy: 0.0


**The good news is that your workflow didn't cause any errors. The bad news is that your model scored the lowest possible accuracy: 0.0! But hey, you just threw away ALL of the text data in the budget. Later, you won't. Before you add the text data, let's see how the model does when scored by log loss.**

## Making predictions

### Use your model to predict values on holdout data

<div class=""><p>You're ready to make some predictions! Remember, the train-test-split you've carried out so far is for model development. The original competition provides an additional test set, for which you'll never actually <em>see</em> the correct labels. This is called the "holdout data."</p>
<p>The point of the holdout data is to provide a fair test for machine learning competitions. If the labels aren't known by anyone but DataCamp, DrivenData, or whoever is hosting the competition, you can be sure that no one submits a mere copy of labels to artificially pump up the performance on their model.</p>
<p>Remember that the original goal is to predict the <strong>probability of each label</strong>. In this exercise you'll do just that by using the <code>.predict_proba()</code> method on your trained model.</p>
<p>First, however, you'll need to load the holdout data, which is available in the workspace as the file <code>HoldoutData.csv</code>.</p></div>

Instructions
<ul>
<li>Read <code>HoldoutData.csv</code> into a DataFrame called <code>holdout</code>. Specify the keyword argument <code>index_col=0</code> in your call to <code>read_csv()</code>.</li>
<li>Generate predictions using <code>.predict_proba()</code> on the numeric columns (available in the <code>NUMERIC_COLUMNS</code> list) of <code>holdout</code>. Make sure to fill in missing values with <code>-1000</code>!</li>
</ul>

In [21]:
# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())

# Fit it to the training data
clf.fit(X_train, y_train)

# Load the holdout data: holdout
holdout = pd.read_csv("https://github.com/lnunesAI/Datacamp/raw/main/3-skill-tracks/case-study-school-budgeting-with-machine-learning-in-python/data/HoldoutData.csv", index_col=0)
holdout = holdout[NUMERIC_COLUMNS].fillna(-1000)

# Generate predictions: predictions
predictions = clf.predict_proba(holdout)

### Writing out your results to a csv for submission

<div class=""><p>At last, you're ready to submit some predictions for scoring. In this exercise, you'll write your predictions to a <code>.csv</code> using the <code>.to_csv()</code> method on a pandas DataFrame. Then you'll evaluate your performance according to the LogLoss metric discussed earlier!</p>
<p>You'll need to make sure your submission obeys the <a href="https://www.drivendata.org/competitions/4/page/15/#sub_values" target="_blank" rel="noopener noreferrer">correct format</a>.</p>
<p>To do this, you'll use your <code>predictions</code> values to create a new DataFrame, <code>prediction_df</code>. </p>
<p><strong>Interpreting LogLoss &amp; Beating the Benchmark:</strong> </p>
<p>When interpreting your log loss score, keep in mind that the score will change based on the number of samples tested. To get a sense of how this <em>very basic</em> model performs, compare your score to the <strong>DrivenData benchmark model performance: 2.0455</strong>, which merely submitted uniform probabilities for each class.</p>
<p>Remember, the lower the log loss the better. Is your model's log loss lower than 2.0455?</p></div>

In [46]:
PATH_TO_PREDICTIONS = 'predictions.csv'
PATH_TO_HOLDOUT_LABELS = 'https://github.com/lnunesAI/Datacamp/raw/main/3-skill-tracks/case-study-school-budgeting-with-machine-learning-in-python/data/TestSetLabelsSample.csv'
BOX_PLOTS_COLUMN_INDICES = [range(0, 37), range(37, 48), range(48, 51), range(51, 76), range(76, 79), range(79, 82), range(82, 87), range(87, 96), range(96, 104)]

In [47]:
test = pd.read_csv(PATH_TO_HOLDOUT_LABELS, index_col=0)
test2 = pd.read_csv(PATH_TO_PREDICTIONS, index_col=0)

In [48]:
def score_submission(pred_path=PATH_TO_PREDICTIONS, holdout_path=PATH_TO_HOLDOUT_LABELS):
    # this happens on the backend to get the score
    holdout_labels = pd.get_dummies(
                        pd.read_csv(holdout_path, index_col=0)
                          .apply(lambda x: x.astype('category'), axis=0)
                      )

    preds = pd.read_csv(pred_path, index_col=0)
    
    # make sure that format is correct
    assert (preds.columns == holdout_labels.columns).all()
    assert (preds.index == holdout_labels.index).all()

    return _multi_multi_log_loss(preds.values, holdout_labels.values)

In [49]:
def _multi_multi_log_loss(predicted,
                          actual,
                          class_column_indices=BOX_PLOTS_COLUMN_INDICES,
                          eps=1e-15):
    """ Multi class version of Logarithmic Loss metric as implemented on
        DrivenData.org
    """
    class_scores = np.ones(len(class_column_indices), dtype=np.float64)

    # calculate log loss for each set of columns that belong to a class:
    for k, this_class_indices in enumerate(class_column_indices):
        # get just the columns for this class
        preds_k = predicted[:, this_class_indices].astype(np.float64)

        # normalize so probabilities sum to one (unless sum is zero, then we clip)
        preds_k /= np.clip(preds_k.sum(axis=1).reshape(-1, 1), eps, np.inf)

        actual_k = actual[:, this_class_indices]

        # shrink predictions so
        y_hats = np.clip(preds_k, eps, 1 - eps)
        sum_logs = np.sum(actual_k * np.log(y_hats))
        class_scores[k] = (-1.0 / actual.shape[0]) * sum_logs

    return np.average(class_scores)

Instructions
<ul>
<li>Create the <code>prediction_df</code> DataFrame by specifying the following arguments to the provided parameters <code>pd.DataFrame()</code>:<ul>
<li><code>pd.get_dummies(df[LABELS]).columns</code>.</li>
<li><code>holdout.index</code>.</li>
<li><code>predictions</code>.</li></ul></li>
<li>Save <code>prediction_df</code> to a csv file called <code>'predictions.csv'</code> using the <code>.to_csv()</code> method.</li>
<li>Submit the predictions for scoring by using the <code>score_submission()</code> function with <code>pred_path</code> set to <code>'predictions.csv'</code>.</li>
</ul>

In [50]:
# Generate predictions: predictions
predictions = clf.predict_proba(holdout[NUMERIC_COLUMNS].fillna(-1000))

# Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS]).columns,
                             index=holdout.index,
                             data=predictions)


# Save prediction_df to csv
prediction_df.to_csv("predictions.csv")

# Submit the predictions for scoring: score
score = score_submission("predictions.csv")

# Print score
print('Your model, trained with numeric data only, yields logloss score: {}'.format(score))

Your model, trained with numeric data only, yields logloss score: 1.9922002790817794


**Even though your basic model scored 0.0 accuracy, it nevertheless performs better than the benchmark score of 2.0455. You've now got the basics down and have made a first pass at this complicated supervised learning problem. It's time to step up your game and incorporate the text data.**

### A very brief introduction to NLP

### Tokenizing text

<div class=""><p>As we talked about in the video, <a href="http://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html" target="_blank" rel="noopener noreferrer">tokenization</a> is the process of chopping up a character sequence into pieces called <em>tokens</em>. </p>
<p>How do we determine what constitutes a token? Often, tokens are separated by whitespace. But we can specify other delimiters as well. For example, if we decided to tokenize on punctuation, then any punctuation mark would be treated like a whitespace. How we tokenize text in our DataFrame can affect the statistics we use in our model.</p>
<p>A particular cell in our budget DataFrame may have the string content <code>Title I - Disadvantaged Children/Targeted Assistance</code>. The number of n-grams generated by this text data is sensitive to whether or not we tokenize on punctuation, as you'll show in the following exercise.</p>
<p>How many tokens (1-grams) are in the string</p>
<pre><code>Title I - Disadvantaged Children/Targeted Assistance
</code></pre>
<p>if we tokenize on whitespace and punctuation?</p></div>

<pre>
Possible Answers
4.
<b>6.</b>
8.
13.
</pre>

**Yes! Tokenizing on whitespace and punctuation means that Children/Targeted becomes two tokens and - is dropped altogether.**

### Testing your NLP credentials with n-grams

<div class=""><p>You're well on your way to NLP superiority. Let's test your mastery of n-grams!</p>
<p>In the workspace, we have the loaded a python list, <code>one_grams</code>, which contains all 1-grams of the string <code>petro-vend fuel and fluids</code>, tokenized on punctuation. Specifically,</p>
<pre><code>one_grams = ['petro', 'vend', 'fuel', 'and', 'fluids']
</code></pre>
<p>In this exercise, your job is to determine the <strong>sum</strong> of the sizes of 1-grams, 2-grams and 3-grams generated by the string <code>petro-vend fuel and fluids</code>, tokenized on punctuation.</p>
<p><em>Recall that the n-gram of a sequence consists of all ordered subsequences of length n.</em></p></div>

In [51]:
one_grams = ['petro', 'vend', 'fuel', 'and', 'fluids']
len(one_grams)+len(one_grams)-1+len(one_grams)-2

12

<pre>
Possible Answers
3.
4.
7.
<b>12.</b>
15.
</pre>

## Representing text numerically

### Creating a bag-of-words in scikit-learn

<div class=""><p>In this exercise, you'll study the effects of tokenizing in different ways by comparing the bag-of-words representations resulting from different token patterns.</p>
<p>You will focus on one feature only, the <code>Position_Extra</code> column, which describes any additional information not captured by the <code>Position_Type</code> label.</p>
<p>For example, in the Shell you can check out the budget item in row 8960 of the data using <code>df.loc[8960]</code>. Looking at the output reveals that this <code>Object_Description</code> is overtime pay. For who? The Position Type is merely "other", but the Position Extra elaborates: "BUS DRIVER". Explore the column further to see more instances. It has a lot of NaN values.</p>
<p>Your task is to turn the raw text in this column into a bag-of-words representation by creating tokens that contain <em>only</em> alphanumeric characters.</p>
<p>For comparison purposes, the first 15 tokens of <code>vec_basic</code>, which splits <code>df.Position_Extra</code> into tokens when it encounters only <em>whitespace</em> characters, have been printed along with the length of the representation.</p></div>

Instructions
<ul>
<li>Import <code>CountVectorizer</code> from <code>sklearn.feature_extraction.text</code>.</li>
<li>Fill missing values in <code>df.Position_Extra</code> using <code>.fillna('')</code> to replace NaNs with empty strings. Specify the additional keyword argument <code>inplace=True</code> so that you don't have to assign the result back to <code>df</code>.</li>
<li>Instantiate the <code>CountVectorizer</code> as <code>vec_alphanumeric</code> by specifying the <code>token_pattern</code> to be <code>TOKENS_ALPHANUMERIC</code>.</li>
<li>Fit <code>vec_alphanumeric</code> to <code>df.Position_Extra</code>.</li>
<li>Hit 'Submit Answer' to see the <code>len</code> of the fitted representation as well as the first 15 elements, and compare to <code>vec_basic</code>.</li>
</ul>

In [52]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create the token pattern: TOKENS_ALPHANUMERIC
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Fill missing values in df.Position_Extra
df.Position_Extra.fillna('', inplace=True)

# Instantiate the CountVectorizer: vec_alphanumeric
vec_alphanumeric = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Fit to the data
vec_alphanumeric.fit(df.Position_Extra)

# Print the number of tokens and first 15 tokens
msg = "There are {} tokens in Position_Extra if we split on non-alpha numeric"
print(msg.format(len(vec_alphanumeric.get_feature_names())))
print(vec_alphanumeric.get_feature_names()[:15])

There are 123 tokens in Position_Extra if we split on non-alpha numeric
['1st', '2nd', '3rd', 'a', 'ab', 'additional', 'adm', 'administrative', 'and', 'any', 'art', 'assessment', 'assistant', 'asst', 'athletic']


**Treating only alpha-numeric characters as tokens gives you a smaller number of more meaningful tokens.**

### Combining text columns for tokenization

<div class=""><p>In order to get a bag-of-words representation for all of the text data in our DataFrame, you must first convert the text data in each row of the DataFrame into a single string.</p>
<p>In the previous exercise, this wasn't necessary because you only looked at one column of data, so each row was already just a single string. <code>CountVectorizer</code> expects each row to just be a single string, so in order to use all of the text columns, you'll need a method to turn a list of strings into a single string.</p>
<p>In this exercise, you'll complete the function definition <code>combine_text_columns()</code>. When completed, this function will convert all training text data in your DataFrame to a single string per row that can be passed to the vectorizer object and made into a bag-of-words using the <code>.fit_transform()</code> method.</p>
<p>Note that the function uses <code>NUMERIC_COLUMNS</code> and <code>LABELS</code> to determine which columns to drop. These lists have been loaded into the workspace.</p></div>

Instructions
<ul>
<li>Use the <code>.drop()</code> method on <code>data_frame</code> with <code>to_drop</code> and <code>axis=</code> as arguments to drop the non-text data. Save the result as <code>text_data</code>.</li>
<li>Fill in missing values (inplace) in <code>text_data</code> with blanks (<code>""</code>), using the <code>.fillna()</code> method.</li>
<li>Complete the <code>.apply()</code> method by writing a lambda function that uses the <code>.join()</code> method to join all the items in a row with a space in between.</li>
</ul>

In [56]:
# Define combine_text_columns()
def combine_text_columns(data_frame, to_drop=NUMERIC_COLUMNS + LABELS):
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # Replace nans with blanks
    text_data.fillna('', inplace=True)
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

### What's in a token?

<div class=""><p>Now you will use <code>combine_text_columns</code> to convert all training text data in your DataFrame to a single vector that can be passed to the vectorizer object and made into a bag-of-words using the <code>.fit_transform()</code> method.</p>
<p>You'll compare the effect of tokenizing using any non-whitespace characters as a token and using only alphanumeric characters as a token.</p></div>

Intructions
<ul>
<li>Import <code>CountVectorizer</code> from <code>sklearn.feature_extraction.text</code>.</li>
<li>Instantiate <code>vec_basic</code> and <code>vec_alphanumeric</code> using, respectively, the <code>TOKENS_BASIC</code> and <code>TOKENS_ALPHANUMERIC</code> patterns.</li>
<li>Create the text vector by using the <code>combine_text_columns()</code> function on <code>df</code>.</li>
<li>Using the <code>.fit_transform()</code> method with <code>text_vector</code>, fit and transform first <code>vec_basic</code> and then <code>vec_alphanumeric</code>. Print the number of tokens they contain.</li>
</ul>

In [57]:
# Import the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Create the basic token pattern
TOKENS_BASIC = '\\S+(?=\\s+)'

# Create the alphanumeric token pattern
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Instantiate basic CountVectorizer: vec_basic
vec_basic = CountVectorizer(token_pattern=TOKENS_BASIC)

# Instantiate alphanumeric CountVectorizer: vec_alphanumeric
vec_alphanumeric = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)

# Create the text vector
text_vector = combine_text_columns(df)

# Fit and transform vec_basic
vec_basic.fit_transform(text_vector)

# Print number of tokens of vec_basic
print("There are {} tokens in the dataset".format(len(vec_basic.get_feature_names())))

# Fit and transform vec_alphanumeric
vec_alphanumeric.fit_transform(text_vector)

# Print number of tokens of vec_alphanumeric
print("There are {} alpha-numeric tokens in the dataset".format(len(vec_alphanumeric.get_feature_names())))

There are 1404 tokens in the dataset
There are 1117 alpha-numeric tokens in the dataset


**Notice that tokenizing on alpha-numeric tokens reduced the number of tokens, just as in the last exercise. We'll keep this in mind when building a better model with the Pipeline object next.**